In [ ]:
import cv2
import dlib
import math
import tensorflow as tf
import keras
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from PIL import Image


image_paths = '/content/drive/MyDrive/Spechio_Project/SCUT-FBP5500_v2/Images'


images = []


for file_name in os.listdir(image_paths):
    file_path = os.path.join(image_paths, file_name)


    if os.path.isfile(file_path) and file_name.lower().endswith(('.jpg', '.jpeg', '.png')):

        image = Image.open(file_path)


        images.append((image,file_name))

In [ ]:
def compute_hu_moments(image):

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Threshold the image to create a binary image
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)


    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Get the largest contour (assumed to be the face)
    largest_contour = max(contours, key=cv2.contourArea)

    # Calculate Hu's moments
    moments = cv2.moments(largest_contour)
    hu_moments = cv2.HuMoments(moments)
    hu_moments = -np.sign(hu_moments) * np.log10(np.abs(hu_moments))


    return hu_moments.flatten()


In [ ]:
moment_data = []


for image, name in images:
    image_array = np.array(image)

    hu_moments = compute_hu_moments(image_array)
    moment_data.append({'Image Name': name, 'HuM1': hu_moments[0],
                         'HuM2': hu_moments[1], 'HuM3': hu_moments[2],
                         'HuM4': hu_moments[3], 'HuM5': hu_moments[4], 'HuM6': hu_moments[5], 'HuM7': hu_moments[6],})




<ipython-input-4-dde678da33cc>:17: RuntimeWarning: divide by zero encountered in log10
  hu_moments = -np.sign(hu_moments) * np.log10(np.abs(hu_moments))
<ipython-input-4-dde678da33cc>:17: RuntimeWarning: invalid value encountered in multiply
  hu_moments = -np.sign(hu_moments) * np.log10(np.abs(hu_moments))


In [ ]:
moment_data

[{'Image Name': 'AM218.jpg',
  'HuM1': 0.6675238639859963,
  'HuM2': 2.7653708536636765,
  'HuM3': 4.571714804500195,
  'HuM4': 4.000544274366291,
  'HuM5': -8.2867318826995,
  'HuM6': -5.386324336202297,
  'HuM7': 10.073108205915165},
 {'Image Name': 'AM1948.jpg',
  'HuM1': 0.619725130165214,
  'HuM2': 4.533180461036448,
  'HuM3': 2.3159883601687774,
  'HuM4': 3.7518702335432783,
  'HuM5': -6.788384201911151,
  'HuM6': 6.921980891942694,
  'HuM7': -7.749265064065186},
 {'Image Name': 'AM1946.jpg',
  'HuM1': 0.6026822976954447,
  'HuM2': 2.7063309851785187,
  'HuM3': 3.690274296531014,
  'HuM4': 3.4113166923204843,
  'HuM5': -6.963418440650302,
  'HuM6': -4.807972615734646,
  'HuM7': -8.073128343380688},
 {'Image Name': 'AM214.jpg',
  'HuM1': 0.5346018131876054,
  'HuM2': 1.7660283062030562,
  'HuM3': 2.8194286678773492,
  'HuM4': 3.346347212245229,
  'HuM5': -7.356072449744249,
  'HuM6': -4.328780200082575,
  'HuM7': 6.432298096361845},
 {'Image Name': 'AM1939.jpg',
  'HuM1': 0.649255

In [ ]:
import pandas as pd
new = pd.DataFrame.from_dict(moment_data)
new

,Image Name,HuM1,HuM2,HuM3,HuM4,HuM5,HuM6,HuM7
0,AM218.jpg,0.667524,2.765371,4.571715,4.000544,-8.286732,-5.386324,10.073108
1,AM1948.jpg,0.619725,4.533180,2.315988,3.751870,-6.788384,6.921981,-7.749265
2,AM1946.jpg,0.602682,2.706331,3.690274,3.411317,-6.963418,-4.807973,-8.073128
3,AM214.jpg,0.534602,1.766028,2.819429,3.346347,-7.356072,-4.328780,6.432298
4,AM1939.jpg,0.649256,3.300918,4.162455,3.640197,-8.067858,-5.647372,7.561663
...,...,...,...,...,...,...,...,...
5496,CF710.jpg,0.720051,2.176763,3.913027,5.043985,-9.737903,6.184300,-9.623109
5497,CF438.jpg,0.613901,1.931299,2.800413,5.061932,-8.993405,-6.029336,10.423076
5498,CF711.jpg,0.720573,2.059365,3.984807,5.525398,-10.643374,-6.851710,-10.325735
5499,CF444.jpg,0.252588,0.626765,1.968653,2.727317,-5.075594,-3.040718,6.511519


In [ ]:
new.to_csv('/content/drive/MyDrive/hu_moments.csv', index=False)